# Techniki programowania - projekt 3

### - Michał Mitura 193209
### - Małgorzata Rogala 198123

#### Zadania:
- [X] Wyświetlanie zadanego sygnału
- [X] Generowanie:
   - [X] DFT
   - [X] odwrotnej DFT
   - [X] sygnałów sinusoidalnych, cosinusoidalnych, piłokształtnych i prostokątnych o zadanej częstotliwości
- [X] zaszumienie sygnału (zadanie nr. 8, specyficzne dla grupy)
- [ ] Eksport za pomocą PyBinda

Wykorzystane biblioteki niestandardowe:
- Matplotplusplus
- AudioFile

## 1. Założenia i ogólna struktura programu:
   
Z racji na niemożliwość zmuszenia PyBinda do działania na komputerach obojga z uczestników grupy, program nie został wyeksportowany jako moduł pythonowy.
Stąd też rozumiemy, że nie możemy załączyć działających snippetów w Jupyter Notebooku.
Projekt funkcjonuje jako zestaw funkcji w pliku .cpp. 
Wszystkie funkcje zostały przetestowane i poprawność wyników została porównana z programami zewnętrznymi (np. matlab dla sygnałów generowanych, audacity dla audio)

W celu przechowywania zadanego sygnału została stworzona struktura ``wave``:

        struct wave {
            vector<double> waveform;
            vector<double> length;
        };
Atrybut ``waveform`` zawiera wysokości (Y) sygnału, a ``length`` jego dziedzinę.

Funkcje zostały podzielone na dwie kategorie i zamknięte w namespace'ach celem "uprzątnięcia":

   **create**, czyli grupa służąca generowaniu struktur ``wave`` według zadanych parametrów
   
        namespace create {
            wave gen_wave(double, double, function<void(wave*, double)>);
            void sin_wave(wave*, double);
            void cos_wave(wave*, double);
            void saw_wave(wave*, double);
            void square_wave(wave*, double);
            void noise_wave(wave*, double);
            wave get_from_audio(string, int);
        }

   **operations** - służące wykonywaniu operacji na istniejących już sygnałach
   
        namespace operations {
            void change_amplitude(wave*, double);
            void add_noise(wave*, double);
            wave get_DFT(wave);
            wave get_IDFT(wave);
            void show_waveform(wave);
        }
 
   *Informacje dodatkowe:*
   Biblioteka AudioFile była załączona tylko lokalnie, ze względu na nieumiejętność dołączenia biblioteki typu header-only przez CMake'a.

## 2. Wyświetlanie sygnału
   Wyświetlanie jest realizowane za pomocą funkcji ``void operations::show_waveform(wave)``.
   Wejście: zadany sygnał, jako obiekt typu ``wave``.
   
        void operations::show_waveform(wave wave_to_plot) {
            plot(wave_to_plot.length, wave_to_plot.waveform);
            show();
        }
   Funkcja uruchamia okno matplota wyświetlające sygnał z wejścia.

## 3. Generowanie sygnałów
Sygnały są generowane przez zestaw funkcji zawartych w przestrzeni nazw ``create``.
Podstawową funkcją w tym zestawie jest ``wave create::gen_wave(double, double, function<void(wave*, double)>, int)``.

        wave create::gen_wave(double frequency, double length, function<void(wave*, double)> func, int samples = 100) {
            wave wave1;
            wave1.length = linspace(0, length);
            func(&wave1, frequency);
            return wave1;
        }
Wejście: częstotliwość sygnału, długość sygnału do wygenerowania, funkcja według której ma być wygenerowany sygnał, ilość próbek. <br>
Działanie funkcji: 
1. Tworzy strukturę ``wave``, mającą przechowywać sygnał
2. Wyznaczenie dziedziny funkcji jako przedział (0, ``length``)
3. Wykonanie funkcji ``function`` jako obliczenie wartości funkcji w dziedzinie.
4. Zwrócenie sygnału na wyjściu

``length`` jest aplikowane przez funkcję ``matplot::linspace`` z 3 parametrem (ilością próbek) jako opcja dla użytkownika (default: 100).

Funkcje wykorzystywane poprzez ``gen_wave`` posiadają 2 wejścia - pierwsze to struktura ``wave`` dla której ma zostać wygenerowane ``waveform`` według odpowiedniej funkcji oraz parametr do wykorzystania przez funkcję. Napisane funkcje to:
- ``void sin_wave(wave*, double)`` - do tworzenia sygnału sinusoidalnego
- ``void cos_wave(wave*, double)`` - do tworzenie sygnału cosinusoidalnego
- ``void saw_wave(wave*, double)`` - do tworzenia sygnału piłokształtnego
- ``void square_wave(wave*, double)`` - do tworzenia sygnału prostokątnego
- ``void noise_wave(wave*, double)`` - do tworzenia sygnału losowego (n.wymagane)

Wszystkie te funkcje są bardzo proste i podobne w budowie, np.:

        void create::sin_wave(wave *target, double frequency) {
            target->waveform = transform(target->length, [frequency](double x){ return sin(x * frequency * 2 * pi);});
        }
W tym przypadku parametr na wejściu jest częstotliwością (jedynym innym zaimplementowanym przedstawieniem jest ``noise_wave``, gdzie odpowiada on za maksymalną amplitudę).
Przekształcenia są wykonywane przez funkcje ``matplot::transform()``.
To rozwiązanie wprowadza bardzo wygodną modularność do systemu generowania sygnałów - wystarczy dodać kolejną funkcję pomocniczą do wykorzystania na wejściu ``gen_wave``.

Wykorzystanie tego systemu "w akcji" wygląda następująco:

        double freq, len;
        cin >> freq >> len;
        wave test = create::gen_wave(freq, len, &create::sin_wave);
Stworzy to sygnał sinusoidalny o częstotliwości ``freq`` i długości ``len``.
Poniżej przykład dla wejść freq = 2, len = 3:
![sinusoida](sinusoida.png "sinusoida dla freq = 2, len = 3")

Dodatkowo została wprowadzona funkcja ``create::get_from_audio(string, int)``.

        wave create::get_from_audio(string path, int sampleNum) {
            wave output;
            AudioFile<double> loadedFile;
            loadedFile.load (path);
            output.length = linspace(0, sampleNum, sampleNum);
            for(int i = 0; i < sampleNum; i++) {
                output.waveform.push_back(loadedFile.samples[0][i]);
            }
            return output;
        }
Wejście: Ścieżka do pliku audio, ilość próbek do pobrania. <br>
Wyjście: Sygnał w postaci n próbek z kanału 0 zadanego pliku audio.
Poniżej przykład dla 40.000 próbek pliku test.wav zawartego w plikach projektu
![test.wav](test_wav.png "40.000 próbek test.wav")


## 4. DFT i IDFT
DFT zostało zaimplementowane w dość prostolinnijnej postaci - z wykorzystaniem klasy ``<complex>`` w celu jego obliczenia.

        wave operations::get_DFT(wave input) {
            wave result;
            result.length = input.length;
            for(int i = 0; i < input.length.size(); i++) {
                complex<double> sum = 0;
                for (int j = 0; j < input.length.size(); j++) {
                    complex<double> exp_holder (0, -2 * pi * i / input.length.size() * j);
                    sum += input.waveform[j] * exp(exp_holder);
                }
                result.waveform.push_back(sum.real());
            }
            return result;
        }
Wejście: Sygnał dla którego ma zostać wykonane DFT
Wyjście: Sygnał będący wynikiem DFT dla zadanego sygnału.
Poniżej wynik tej operacji dla wcześniej pokazanego sygnału sinusoidalnego o częstotliwości 2 i długości 3:
![DFT](dft.png "DFT sinusoidy o czéstotliwości 2 i długości 3")

IDFT jest wykonane analogicznie:

        wave operations::get_IDFT(wave input) {
            wave result;
            result.length = input.length;
            for(int i = 0; i < input.length.size(); i++) {
                complex<double> sum = 0;
                for (int j = 0; j < input.length.size(); j++) {
                    complex<double> exp_holder (0, 2 * pi * i / input.length.size() * j);
                    sum += input.waveform[j] * exp(exp_holder);
                }
                result.waveform.push_back(sum.real() / result.length.size());
            }
            return result;
        }
poniżej wyjście dla takiego samego sygnału:
![IDFT](IDFT.png "IDFT sinusoidy o czéstotliwości 2 i długości 3")

W ramach testu zostało też przetestowane wykonanie w DFT i IDFT na tym samym sygnale kodem jak poniżej:

        wave test = create::gen_wave(2, 3, &create::sin_wave);
        wave dft = operations::get_DFT(test);
        wave idft = operations::get_IDFT(dft);
        operations::show_waveform(idft);
Zwraca on oryginalną funkcję, wykres poniżej:
![DFT-IDFT](DFT-IDFT.png "sygnał sinusoidalny (częst. 2, dł. 3) na którym przeprowadzono kolejno DFT i IDFT")

## 5. Zaszumianie sygnału (zadanie 8)
Zaszumianie zostało wykonane w podobny sposób do generowania sygnałów - korzysta z funkcji ``matplot::transform()`` z zastosowaniem ``matplot::rand()``

        void operations::add_noise(wave* target, double amount) {
            target->waveform = transform(target->waveform, [=](double x){ return x + rand( -amount , amount);});
        }
Wejście: sygnał do zaszumienia, ilość szumu (jego amplituda) <br>
Działanie: Do każdej próbki sygnału jest dodawana losowa wartość z zakresu (-x, x), gdzie x jest podane przez użytkownika
Przykład dla wcześniejszej sinusoidy i wartości amplitudy 1:
![zaszumione](szum.png "zaszumiona sinusoida")

## 6. Funkcjonalności dodatkowe
Dodatkowo (poza zadaniami wymaganymi) zostały wykonane funkcje:
- ``void operations::change_amplitude(wave*, double)`` - zmieniająca amplitudę sygnału
- ``void create::noise_wave(wave*, double)`` - tworząca sygnał losowy o zadanej amplitudzie